<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Product_Marketing_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Sample Data for AI Product Marketing System for Image Generation
# Product Marketing AI System

**Overview:**

The Product Marketing AI System automatically creates high-quality marketing visuals by processing, enhancing, and refining input images.

**Key Steps:**
- **Input & Preprocessing:**  
  Users provide a base image (e.g., a background or scene) and a secondary image (e.g., a product or model). The system enhances these images, adjusts resolution and contrast, and generates segmentation masks.

- **Context & Prompt Generation:**  
  It analyzes the images to extract contextual details and automatically generates a detailed, structured marketing prompt tailored to the product and target audience.

- **Image Generation & Refinement:**  
  Advanced AI models use the prompt to generate photorealistic marketing images. The system then refines the results based on user feedback to ensure the output meets professional quality standards.

- **Quality Evaluation:**  
  Final images are evaluated using metrics (such as SSIM, PSNR, and color histograms) to compare them with the original inputs. Additionally, AI-driven feedback helps further improve image quality.

**Applications:**
- **Digital Advertising & E-Commerce:**  
  Generate stunning visuals for online stores, social media campaigns, and ad banners.
- **Branding & Marketing:**  
  Enhance and standardize visual branding materials across various industries such as fashion, tech, automotive, and real estate.
- **Content Creation:**  
  Streamline production of professional digital content for websites, promotions, and digital signage.

This system provides a seamless, automated solution for creating visually appealing marketing assets, reducing manual effort and ensuring consistency and high quality across all outputs.

In [1]:
!pip install serpapi google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=f9a3ee26b8f75ae29350ad2721b9079eb5c5bbedbc52164011a906b0d01d813d
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results


In [11]:
import os
import requests
import json
import uuid
import csv
import time
import re
import random
from urllib.parse import urljoin, urlparse
from bs4 import BeautifulSoup
from PIL import Image
import numpy as np
import cv2
from serpapi import GoogleSearch

BASE_DIR = "ai_marketing_test_dataset"
BASE_IMAGES_FOLDER = os.path.join(BASE_DIR, "base_images")
SECONDARY_IMAGES_FOLDER = os.path.join(BASE_DIR, "secondary_images")
METADATA_FILE = os.path.join(BASE_DIR, "image_metadata.csv")

os.makedirs(BASE_IMAGES_FOLDER, exist_ok=True)
os.makedirs(SECONDARY_IMAGES_FOLDER, exist_ok=True)

SERPAPI_KEY = "95c2797a69b167639c98ab054e8597d752c6fe6d"

TARGET_VALID_IMAGES_PER_QUERY = 25
SERPAPI_NUM_PER_PAGE = 100
MAX_PAGES_PER_QUERY = 7
API_CALL_DELAY_SECONDS = 5
PAGE_DELAY_SECONDS = 3
DOWNLOAD_DELAY_SECONDS = 0.7
MAX_DOWNLOAD_SIZE_MB = 15

MIN_DIMENSION = 600
MIN_VARIANCE = 50
MIN_LAPLACIAN = 25

In [12]:
INDUSTRY_TEMPLATES = {
    "fashion": {
        "base_queries": [
            "fashion photoshoot background outdoor city",
            "luxury retail store interior clean",
            "urban street fashion backdrop bokeh",
            "minimalist white studio background fashion",
            "runway show background lights",
            "vintage boutique interior",
            "beach fashion shoot background",
            "mountain landscape fashion backdrop"
        ],
        "secondary_queries": [
            "female fashion model standing studio",
            "male fashion model walking urban",
            "diverse group of fashion models laughing",
            "plus size fashion model portrait",
            "model wearing casual outfit outdoor",
            "model wearing formal dress red carpet",
            "model wearing activewear gym",
            "model close up face beauty shot",
            "model full body shot isolated",
            "designer dress on hanger studio",
            "luxury handbag closeup on table",
            "fashion accessories collection white background",
            "luxury watch on wrist close up detail",
            "elegant necklace on display stand",
            "stylish mens shirt studio shot",
            "fashion trousers folded white background",
            "denim jeans stack photography",
            "pair of sneakers clean background",
            "high heels studio shot",
            "winter coat on mannequin",
            "swimsuit on model beach",
            "sunglasses on face close up",
            "scarf draped elegant",
            "leather wallet photography"
        ]
    },
    "food": {
        "base_queries": [
            "gourmet kitchen countertop high angle marble",
            "fine dining restaurant table setting",
            "rustic wooden table background bokeh",
            "outdoor cafe patio sunny",
            "bakery display case vibrant",
            "farmers market produce display",
            "cozy home kitchen window light"
        ],
        "secondary_queries": [
            "plated gourmet dish close up steak",
            "appetizing dessert photography chocolate cake",
            "fresh fruit and vegetable colorful arrangement",
            "refreshing cocktail condensation glass bar",
            "coffee cup steam close up",
            "artisanal bread on cutting board",
            "soup bowl overhead shot",
            "pizza slice melted cheese close up",
            "sushi platter arrangement",
            "bottle of wine with glass",
            "ice cream scoop texture",
            "raw ingredients selection herbs spices"
        ]
    },
    "tech": {
        "base_queries": [
            "modern office desk setup laptop monitor",
            "minimalist white workspace clean",
            "futuristic tech lab abstract",
            "urban technology street view night",
            "server room blue light",
            "gaming setup neon lights",
            "smart home interior blurred background"
        ],
        "secondary_queries": [
            "smartphone screen close up hand holding",
            "laptop on modern desk professional setting",
            "smartwatch on wrist active",
            "wireless earbuds charging case open",
            "mechanical keyboard close up",
            "gaming mouse and pad",
            "tablet device in hands using stylus",
            "portable speaker outdoor nature",
            "noise cancelling headphones photography",
            "dslr camera lens detail",
            "drone flying over landscape",
            "smart home speaker on counter",
            "external hard drive photography"
        ]
    },
    "beauty": {
        "base_queries": [
            "luxury bathroom vanity clean marble",
            "serene spa setting with candles plants",
            "natural light portrait background soft focus",
            "elegant dressing table makeup setup",
            "cosmetic store aisle blurred",
            "tropical resort spa background"
        ],
        "secondary_queries": [
            "model with clear skin glowing portrait",
            "makeup palette brushes flat lay",
            "perfume bottle luxury gold",
            "skincare product texture cream swatch",
            "hair product shampoo bottle studio",
            "model applying lipstick mirror",
            "model demonstrating face serum application",
            "nail polish bottles colorful",
            "cosmetic brush set clean background",
            "bar of soap natural ingredients",
            "essential oil bottle with plants"
        ]
    },
    "automotive": {
        "base_queries": [
            "scenic coastal road sunset drive",
            "city parking lot night lights",
            "clean automotive showroom floor",
            "empty desert road daytime",
            "snowy mountain road empty",
            "industrial warehouse interior"
        ],
        "secondary_queries": [
            "luxury sedan studio shot low angle",
            "sports car dynamic action shot race track",
            "rugged SUV off-road trail",
            "car interior dashboard detail modern",
            "motorcycle parked city street",
            "electric car charging station futuristic",
            "vintage car restored sunny day",
            "car wheel rim close up",
            "headlights detail night",
            "convertible car open top"
        ]
    },
    "real_estate": {
        "base_queries": [
            "modern living room interior spacious light",
            "luxury kitchen interior island",
            "house exterior sunny day landscaped yard",
            "apartment balcony view cityscape high rise",
            "backyard garden landscape patio",
            "bedroom interior cozy minimalist",
            "modern bathroom interior clean",
            "home office setup window view",
            "building facade modern architecture",
            "rooftop terrace city view"
        ],
        "secondary_queries": [
            "luxury sofa in living room",
            "decorative abstract art on wall",
            "modern kitchen appliance set stainless steel",
            "happy family relaxing on sofa",
            "person enjoying balcony view coffee",
            "dining table setup elegant",
            "comfortable armchair reading nook",
            "exterior door entryway welcoming",
            "swimming pool luxury backyard",
            "patio furniture setup"
        ]
    },
    "travel": {
        "base_queries": [
            "tropical beach paradise sunset",
            "mountain landscape panorama",
            "historic city street europe",
            "luxury hotel lobby elegant",
            "airplane window view clouds",
            "cruise ship deck ocean",
            "safari jeep african savannah",
            "northern lights night sky"
        ],
        "secondary_queries": [
            "traveler with backpack hiking trail",
            "couple holding hands beach sunset",
            "family vacation photo happy",
            "suitcase and passport travel",
            "airplane wing view clouds",
            "hotel room modern",
            "travel guidebook and map",
            "tourist taking photo landmark",
            "adventure gear camping",
            "local cuisine street food market"
        ]
    },
    "health": {
        "base_queries": [
            "modern hospital lobby clean",
            "doctor office examination room",
            "fitness center gym equipment",
            "yoga studio calm",
            "pharmacy interior shelves",
            "medical lab research",
            "physical therapy clinic",
            "dental office waiting room"
        ],
        "secondary_queries": [
            "doctor in white coat stethoscope",
            "nurse caring for patient",
            "fitness trainer helping client",
            "yoga instructor demonstrating pose",
            "pharmacist behind counter",
            "medical researcher in lab",
            "physical therapist assisting patient",
            "dentist examining patient",
            "healthy meal prep ingredients",
            "exercise equipment home gym"
        ]
    },
    "education": {
        "base_queries": [
            "modern classroom whiteboard",
            "university lecture hall students",
            "school library bookshelves",
            "science lab experiment",
            "art classroom creative",
            "school playground children playing",
            "online learning virtual classroom",
            "study group students notebooks"
        ],
        "secondary_queries": [
            "teacher explaining lesson",
            "student studying book",
            "graduation ceremony cap and gown",
            "science experiment equipment",
            "art supplies paintbrushes",
            "children playing on playground",
            "laptop online course",
            "study notes and textbooks",
            "school bus yellow",
            "university campus buildings"
        ]
    }
}

GENERAL_QUERIES = {
    "base_queries": [
        "clean white seamless background high resolution studio",
        "gradient studio background soft colors pastel",
        "abstract geometric background modern",
        "blurred bokeh light background warm",
        "minimalist empty room background concrete",
        "simple colored paper background",
        "texture background concrete wall",
        "wooden surface background tabletop"
    ],
    "secondary_queries": [
        "isolated product on transparent background mockup",
        "person full body pose white background standing",
        "still life objects arrangement clean background",
        "single item close up studio white background",
        "cutout person on white background",
        "isolated electronics white background",
        "isolated food item white background",
        "isolated fashion item white background",
        "isolated accessory white background",
        "group of diverse people white background casual"
    ]
}

In [13]:
def init_metadata():
    if not os.path.exists(METADATA_FILE):
        with open(METADATA_FILE, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(["image_path", "category", "source_url", "industry", "type", "source_query"])
        print(f"Initialized metadata file: {METADATA_FILE}")
    else:
        print(f"Metadata file already exists: {METADATA_FILE}. Appending data.")

def is_valid_url(url):
    if not isinstance(url, str):
        return False
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

def perform_quality_check(image_path):
    try:
        with Image.open(image_path) as img:
            if img.size[0] < MIN_DIMENSION or img.size[1] < MIN_DIMENSION:
                return False
        img_cv = cv2.imread(image_path)
        if img_cv is None:
            return False
        if img_cv.shape[0] < 3 or img_cv.shape[1] < 3 or np.prod(img_cv.shape[:2]) < 100:
            return False
        img_cv_gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)
        variance = np.var(img_cv_gray)
        if variance < MIN_VARIANCE:
            return False
        laplacian = cv2.Laplacian(img_cv_gray, cv2.CV_64F).var()
        if laplacian < MIN_LAPLACIAN:
            return False
        return True
    except Exception:
        return False

def download_image(url, folder, category, industry, image_type, source_query):
    if not is_valid_url(url):
        return None
    if re.search(r'\.(svg|gif)$', url.lower()) or "base64," in url.lower() or url.endswith('/') or url.endswith('#') or 'placeholder' in url.lower():
        return None
    save_path = None
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        response = requests.get(url, stream=True, timeout=45, headers=headers)
        response.raise_for_status()
        content_type = response.headers.get('content-type', '').lower()
        ext = ".jpg"
        if 'jpeg' in content_type:
            ext = ".jpg"
        elif 'png' in content_type:
            ext = ".png"
        elif 'image/' in content_type:
            url_path = urlparse(url).path
            url_ext = os.path.splitext(url_path)[1].lower()
            if url_ext in ['.jpg', '.jpeg', '.png']:
                ext = url_ext
            elif 'gif' in url_ext or 'svg' in url_ext:
                response.close()
                return None
            else:
                response.close()
                return None
        else:
            response.close()
            return None
        image_name = f"{uuid.uuid4()}{ext}"
        save_path = os.path.join(folder, image_name)
        downloaded_size = 0
        max_bytes = MAX_DOWNLOAD_SIZE_MB * 1024 * 1024
        with open(save_path, "wb") as f:
            for chunk in response.iter_content(8192):
                f.write(chunk)
                downloaded_size += len(chunk)
                if downloaded_size > max_bytes:
                    response.close()
                    os.remove(save_path)
                    return None
        response.close()
        if perform_quality_check(save_path):
            with open(METADATA_FILE, "a", newline="", encoding="utf-8") as f:
                writer = csv.writer(f)
                writer.writerow([save_path, category, url, industry, image_type, source_query])
            return save_path
        else:
            if os.path.exists(save_path):
                os.remove(save_path)
            return None
    except requests.exceptions.RequestException:
        if save_path and os.path.exists(save_path):
            try:
                os.remove(save_path)
            except OSError:
                pass
        return None
    except Exception:
        if save_path and os.path.exists(save_path):
            try:
                os.remove(save_path)
            except OSError:
                pass
        return None

def google_image_search(query, max_results=TARGET_VALID_IMAGES_PER_QUERY, max_pages=MAX_PAGES_PER_QUERY):
    if not SERPAPI_KEY or SERPAPI_KEY == "YOUR_SERPAPI_KEY":
        print("SerpAPI key not configured or is placeholder. Skipping Google Image Search.")
        return []
    print(f"Searching Google Images for: '{query}' (targeting up to {max_results} results across {max_pages} pages)")
    all_image_urls = set()
    page_num = 0
    retries_left = 3
    while len(all_image_urls) < max_results * 2 and page_num < max_pages:
        print(f"  Fetching page {page_num + 1}/{max_pages} (collected {len(all_image_urls)} URLs so far)...")
        params = {
            "q": query,
            "tbm": "isch",
            "ijn": str(page_num),
            "api_key": SERPAPI_KEY,
            "safe": "active",
            "num": SERPAPI_NUM_PER_PAGE
        }
        try:
            search = GoogleSearch(params)
            results = search.get_dict()
            if results.get("error"):
                error_msg = results["error"]
                print(f"  SerpAPI error for '{query}' (page {page_num + 1}): {error_msg}")
                if "invalid api key" in error_msg.lower():
                    print("  SERPAPI_KEY seems invalid. Please check your key.")
                    return list(all_image_urls)
                elif "daily limit exceeded" in error_msg.lower() or "account has insufficient funds" in error_msg.lower() or "quota" in error_msg.lower():
                    print("  SERPAPI limit/quota issue. Stopping search.")
                    return list(all_image_urls)
                else:
                    retries_left -= 1
                    if retries_left > 0:
                        print(f"  Retrying page {page_num + 1} ({retries_left} retries left)...")
                        time.sleep(API_CALL_DELAY_SECONDS + random.uniform(0, 2))
                        continue
                    else:
                        print(f"  Max retries reached for page {page_num + 1}. Stopping search for this query.")
                        break
            images_results = results.get("images_results", [])
            if not images_results:
                print(f"  No results found on page {page_num + 1} or end of results reached.")
                break
            current_page_urls = []
            for img in images_results:
                url = img.get("original") or img.get("link")
                if url and is_valid_url(url):
                    if url not in all_image_urls:
                        all_image_urls.add(url)
                        current_page_urls.append(url)
            print(f"  Added {len(current_page_urls)} new potential URLs from page {page_num + 1}. Total unique URLs collected: {len(all_image_urls)}")
            if len(images_results) < SERPAPI_NUM_PER_PAGE:
                print(f"  Page {page_num + 1} returned less than {SERPAPI_NUM_PER_PAGE} results, likely the last page.")
                break
            page_num += 1
            retries_left = 3
            if len(all_image_urls) < max_results * 2 and page_num < max_pages:
                time.sleep(PAGE_DELAY_SECONDS + random.uniform(0, 1))
        except Exception as e:
            print(f"SerpAPI library error for '{query}' (page {page_num + 1}): {e}")
            error_str = str(e).lower()
            if "invalid api key" in error_str:
                print("  SERPAPI_KEY seems invalid. Please check your key.")
                break
            elif "daily limit exceeded" in error_str or "account has insufficient funds" in error_str or "quota" in error_str:
                print("  SERPAPI limit/quota issue. Stopping search.")
                break
            else:
                retries_left -= 1
                if retries_left > 0:
                    print(f"  Retrying page {page_num + 1} ({retries_left} retries left)...")
                    time.sleep(API_CALL_DELAY_SECONDS + random.uniform(0, 2))
                    continue
                else:
                    print(f"  Max retries reached for page {page_num + 1}. Stopping search for this query.")
                    break
    print(f"Finished searching for '{query}'. Total unique URLs collected: {len(all_image_urls)}")
    return list(all_image_urls)

def create_category_name(query, industry, image_type):
    words = query.lower().split()
    meaningful_words = [word for word in words if len(word) > 2 and word not in ['a', 'an', 'the', 'in', 'on', 'with', 'for', 'and', 'high', 'low', 'up', 'down', 'clean', 'white', 'black', 'red', 'blue', 'studio', 'background', 'image', 'photo', 'photography', 'picture', 'view', 'shot']]
    base_name = "_".join(meaningful_words[:4])
    if not base_name:
        base_name = "misc"
    base_name = re.sub(r'[^a-z0-9_]', '', base_name)
    base_name = re.sub(r'_{2,}', '_', base_name).strip('_')
    return f"{industry}_{image_type}_{base_name}"

In [ ]:
def main():
    if not SERPAPI_KEY or SERPAPI_KEY == "YOUR_SERPAPI_KEY":
        print("="*70)
        print("!!! SERPAPI_KEY is not set or is the placeholder. !!!")
        print("!!! Please replace it with your actual, valid SerpAPI key. !!!")
        print("!!! You can get one at: https://serpapi.com/users/sign_up      !!!")
        print("!!! Image search will be skipped.                          !!!")
        print("="*70)
        init_metadata()
        summary = {
            "base_images_downloaded": 0,
            "secondary_images_downloaded": 0,
            "total_images_downloaded": 0,
            "base_image_paths_sample": [],
            "secondary_image_paths_sample": []
        }
        summary_file_path = os.path.join(BASE_DIR, "download_summary.json")
        with open(summary_file_path, "w", encoding="utf-8") as f:
            json.dump(summary, f, indent=4)
        print("\n--- Dataset Building Skipped (No API Key) ---")
        print(f"Summary saved to: {summary_file_path}")
        print(json.dumps(summary, indent=4))
        print(f"Detailed metadata file initialized (empty): {METADATA_FILE}")
        return
    init_metadata()
    image_sources_list = []
    for industry, data in INDUSTRY_TEMPLATES.items():
        for query in data.get("base_queries", []):
            category = create_category_name(query, industry, "base")
            image_sources_list.append(("google", query, category, industry, "base"))
        for query in data.get("secondary_queries", []):
            category = create_category_name(query, industry, "secondary")
            image_sources_list.append(("google", query, category, industry, "secondary"))
    for query in GENERAL_QUERIES.get("base_queries", []):
        category = create_category_name(query, "general", "base")
        image_sources_list.append(("google", query, category, "various", "base"))
    for query in GENERAL_QUERIES.get("secondary_queries", []):
        category = create_category_name(query, "general", "secondary")
        image_sources_list.append(("google", query, category, "various", "secondary"))
    random.shuffle(image_sources_list)
    all_downloaded_paths = {"base": [], "secondary": []}
    print(f"Starting image download process.")
    print(f"Targeting approx. {TARGET_VALID_IMAGES_PER_QUERY} valid images per source query.")
    print(f"Defined {len(image_sources_list)} distinct source queries.")
    for i, (source_type, source_query, category, industry, image_type) in enumerate(image_sources_list):
        print(f"\n--- Processing Query {i+1}/{len(image_sources_list)} ---")
        print(f"Type: {image_type}, Industry: '{industry}', Category: '{category}'")
        folder = BASE_IMAGES_FOLDER if image_type == "base" else SECONDARY_IMAGES_FOLDER
        downloaded_count_for_source = 0
        attempted_urls_count = 0
        if source_type == "google":
            potential_image_urls = google_image_search(
                source_query,
                max_results=TARGET_VALID_IMAGES_PER_QUERY,
                max_pages=MAX_PAGES_PER_QUERY
            )
            if not potential_image_urls:
                print(f"No image URLs found for query: '{source_query}' after searching pages {MAX_PAGES_PER_QUERY}.")
                time.sleep(API_CALL_DELAY_SECONDS + random.uniform(0, 1))
                continue
            print(f"Found {len(potential_image_urls)} potential URLs. Attempting to download and validate...")
            for url in potential_image_urls:
                if downloaded_count_for_source >= TARGET_VALID_IMAGES_PER_QUERY:
                    print(f"Reached target count ({TARGET_VALID_IMAGES_PER_QUERY}) for query '{source_query}'. Stopping downloads for this query.")
                    break
                attempted_urls_count += 1
                path = download_image(url, folder, category, industry, image_type, source_query)
                if path:
                    all_downloaded_paths[image_type].append(path)
                    downloaded_count_for_source += 1
                    print(f"  Successfully downloaded & validated ({downloaded_count_for_source}/{TARGET_VALID_IMAGES_PER_QUERY}): {os.path.basename(path)}")
                time.sleep(DOWNLOAD_DELAY_SECONDS + random.uniform(0, 0.2))
            print(f"Finished processing query '{source_query}'. Downloaded {downloaded_count_for_source} valid images.")
        time.sleep(API_CALL_DELAY_SECONDS + random.uniform(0, 1.5))
    summary = {
        "base_images_downloaded": len(all_downloaded_paths["base"]),
        "secondary_images_downloaded": len(all_downloaded_paths["secondary"]),
        "total_images_downloaded": len(all_downloaded_paths["base"]) + len(all_downloaded_paths["secondary"]),
        "base_image_paths_sample": all_downloaded_paths["base"][:10],
        "secondary_image_paths_sample": all_downloaded_paths["secondary"][:10]
    }
    summary_file_path = os.path.join(BASE_DIR, "download_summary.json")
    with open(summary_file_path, "w", encoding="utf-8") as f:
        json.dump(summary, f, indent=4)
    print("\n--- Dataset Building Complete ---")
    print(f"Summary saved to: {summary_file_path}")
    print(json.dumps(summary, indent=4))
    print(f"Detailed metadata saved to: {METADATA_FILE}")
    print(f"Images saved to: {BASE_IMAGES_FOLDER} and {SECONDARY_IMAGES_FOLDER}")
    print(f"Total queries processed: {len(image_sources_list)}")

if __name__ == "__main__":
    main()

Metadata file already exists: ai_marketing_test_dataset/image_metadata.csv. Appending data.
Starting image download process.
Targeting approx. 25 valid images per source query.
Defined 196 distinct source queries.

--- Processing Query 1/196 ---
Type: secondary, Industry: 'tech', Category: 'tech_secondary_external_hard_drive'
Searching Google Images for: 'external hard drive photography' (targeting up to 25 results across 7 pages)
  Fetching page 1/7 (collected 0 URLs so far)...
  SerpAPI error for 'external hard drive photography' (page 1): Invalid API key. Your API key should be here: https://serpapi.com/manage-api-key
  SERPAPI_KEY seems invalid. Please check your key.
No image URLs found for query: 'external hard drive photography' after searching pages 7.

--- Processing Query 2/196 ---
Type: secondary, Industry: 'fashion', Category: 'fashion_secondary_diverse_group_fashion_models'
Searching Google Images for: 'diverse group of fashion models laughing' (targeting up to 25 results 